In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import random 
import h5py
import cv2
import glob
from matplotlib import pyplot

from keras.models import Sequential 
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard
import os
from mtcnn.mtcnn import MTCNN
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import load_model 
from decimal import *
from matplotlib import pyplot
import keras.callbacks

Using TensorFlow backend.


In [2]:
# label = ['bale','cristiano','modric','benzema','casemero','carvajall','kross','iscoo','ramos','marcelo','cortoiss','hazardd']

In [3]:
label = ['bale','benzema','carvajal','casemero','cortoiss','cristiano','hazardd','isco','kross','marcelo','messi','modric','nacho','navas','pepe','ramos','valverde','varane','vasquiz']
path = {}
nb_train_samples = 0
for img in label:
    path[img] = glob.glob(os.path.join('trainRealMadrid/', img, '*'))
    print(img,len(path[img]))
    nb_train_samples += len(path[img])
print(nb_train_samples)

bale 192
benzema 225
carvajal 116
casemero 244
cortoiss 148
cristiano 350
hazardd 238
isco 146
kross 153
marcelo 177
messi 72
modric 192
nacho 195
navas 165
pepe 89
ramos 213
valverde 76
varane 72
vasquiz 43
3106


In [4]:
# img_width=200
# img_height=200
img_width=128
img_height=128

In [5]:
path = {}
X_train=[]
Y_train=[]
classe=0
for img in label:
    path[img] = glob.glob(os.path.join('trainRealMadrid', img, '*'))
    for image in path[img]:
        X_train.append(cv2.resize(cv2.imread(image),(img_width,img_width)))
        Y_train.append(classe)
    classe=classe+1
    print(len(path[img]))

192
225
116
244
148
350
238
146
153
177
72
192
195
165
89
213
76
72
43


In [ ]:
cv2.imshow("k",X_train[-1])
cv2.waitKey(0)


In [8]:
batch_size=64
input_shape=(img_width,img_width,3)
path='train'
epochs=5
X_train =  np.array(X_train[:])
X_train =  X_train / 255

In [7]:
print(len(X_train))
print(len(Y_train))

3106
3106


# parametre de model

In [8]:
# cnn_model = Sequential()

# cnn_model.add(Conv2D(64, 6, 6, input_shape = (128, 128, 3), activation = 'relu'))
# cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

# cnn_model.add(Dropout(0.2))

# cnn_model.add(Conv2D(64, 5, 5, activation = 'relu'))
# cnn_model.add(MaxPooling2D(pool_size = (2, 2)))

# cnn_model.add(Flatten())

# cnn_model.add(Dense(output_dim = 128, activation = 'relu'))

# cnn_model.add(Dense(output_dim = 1, activation = 'sigmoid'))
# cnn_model.summary()

In [9]:
cnn_model = Sequential()
cnn_model.add(Conv2D(input_shape=input_shape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Conv2D(filters=1024, kernel_size=(3,3), padding="same", activation="relu"))
cnn_model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
cnn_model.add(Dropout(0.2))

cnn_model.add(Flatten())
cnn_model.add(Dense(units=4096,activation="relu"))
cnn_model.add(Dense(units=2048,activation="relu"))
cnn_model.add(Dropout(0.2))

cnn_model.add(Dense(units=len(label), activation="softmax"))
cnn_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 64)      1792      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 128)       0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 256)      

In [9]:
# cnn_model = Sequential()

# cnn_model.add(Conv2D(32, (3,3), input_shape = input_sape, activation = 'relu'))
# cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
# cnn_model.add(Dropout(0.2))


# cnn_model.add(Conv2D(32, (3,3), activation = 'relu'))
# cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Conv2D(64, (3,3), activation = 'relu'))
# cnn_model.add(MaxPooling2D(pool_size = (2, 2)))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Flatten())

# cnn_model.add(Dense(units = 4096, activation = 'relu'))
# cnn_model.add(Dense(units = 2048, activation = 'relu'))
# cnn_model.add(Dropout(0.2))

# cnn_model.add(Dense(units = len(label), activation = 'softmax'))


# cnn_model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 198, 198, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 99, 99, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 99, 99, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 97, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 48, 48, 32)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 46, 64)       

In [10]:
cnn_model.compile(loss = 'sparse_categorical_crossentropy', optimizer ='adam', metrics = ['accuracy'])

In [11]:
ACCURACY_THRESHOLD = 0.99
class MyCustomCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if(logs.get('accuracy') > ACCURACY_THRESHOLD):   
            print("\nReached %2.2f%% accuracy, so stopping training!!" %(ACCURACY_THRESHOLD*100))   
            self.model.stop_training = True

callback = MyCustomCallback()

In [12]:
history = cnn_model.fit(X_train, Y_train, batch_size =batch_size, nb_epoch = 70, verbose = 1,callbacks=[callback])
#,validation_split=0.1

C:\Users\Public\anaconda\lib\site-packages\ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Epoch 1/70
3106/3106 [==============================] - 840s 270ms/step - loss: 3.4171 - accuracy: 0.1043
Epoch 2/70
3106/3106 [==============================] - 777s 250ms/step - loss: 2.8487 - accuracy: 0.1127
Epoch 3/70
3106/3106 [==============================] - 831s 268ms/step - loss: 2.8493 - accuracy: 0.1124
Epoch 4/70
3106/3106 [==============================] - 857s 276ms/step - loss: 2.8483 - accuracy: 0.1127
Epoch 5/70
3106/3106 [==============================] - 840s 271ms/step - loss: 2.7196 - accuracy: 0.1336
Epoch 6/70
3106/3106 [==============================] - 815s 262ms/step - loss: 2.5459 - accuracy: 0.1710
Epoch 7/70
3106/3106 [==============================] - 749s 241ms/step - loss: 2.1951 - accuracy: 0.2627
Epoch 8/70
3106/3106 [==============================] - 561s 181ms/step - loss: 1.8865 - accuracy: 0.3693
Epoch 9/70
3106/3106 [==============================] - 509s 164ms/step - loss: 1.5958 - accuracy: 0.4755
Epoch 10/70
3106/3106 [=======================

In [13]:
cnn_model.save('t8.h5')

In [ ]:
cnn_model = load_model('t8.h5')

In [27]:
filename = 'test.jpg'
img=cv2.imread(filename)
pixels = pyplot.imread(filename)
detector = MTCNN()
predict=label
result = detector.detect_faces(pixels)
if result != []: 
    for person in result:
        bounding_box = person['box']
        f = cv2.resize(img[bounding_box[1]:bounding_box[1] + bounding_box[3],bounding_box[0]:bounding_box[0]+bounding_box[2]],(img_width,img_width))
        k=f
        f = np.reshape(f,[1,img_width,img_width,3])
        f = f / 255
        classes = cnn_model.predict_classes(f)[0]
        name=predict[classes]
        prob=cnn_model.predict_proba(f)[0][classes]
        color=(0,255,0)#green
        if prob<0.5:
            name='unkonown'
            color=(0,0,255)#red
        cv2.rectangle(img,
                      (bounding_box[0], bounding_box[1]+5),
                      (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                      color,2) 
        cv2.putText(img,name,(bounding_box[0]-6, bounding_box[1]-11), 1, 1,color,2,cv2.LINE_AA)
        cv2.putText(img," "+str(Decimal(str(prob)).quantize(Decimal((0, (1,), -2)), rounding=ROUND_DOWN)),(bounding_box[0]-9, bounding_box[1]+2), 1, 1,color,2,cv2.LINE_AA)
       
    cv2.imshow("Image", cv2.resize(img,(940,600)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
predict=label
cap = cv2.VideoCapture(0)
numimg = 207 
while cap.isOpened():
    ret, img = cap.read()   
    detector = MTCNN()
    result = detector.detect_faces(img)
    if result != []: 
        for person in result:
            bounding_box = person['box']
            try:
                f = cv2.resize(img[bounding_box[1]:bounding_box[1] + bounding_box[3],bounding_box[0]:bounding_box[0]+bounding_box[2]],(128,128))
                f = np.reshape(f,[1,128,128,3])
                f = f / 255
                classes = cnn_model.predict_classes(f)[0]
                name=predict[classes]
                prob=cnn_model.predict_proba(f)[0][classes]
                color=(0,255,0)#green
                if prob<0.1:
                    name='unkonown'
                    color=(0,0,255)#red
                cv2.rectangle(img,
                              (bounding_box[0], bounding_box[1]),
                              (bounding_box[0]+bounding_box[2], bounding_box[1] + bounding_box[3]),
                              color,2) 
                cv2.putText(img,name+" "+str(Decimal(str(prob)).quantize(Decimal((0, (1,), -2)), rounding=ROUND_DOWN)),(bounding_box[0], bounding_box[1]-6), 1, 1,color,2,cv2.LINE_AA)
            except:
                pass
    cv2.imshow("Image", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()